In [ ]:
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=False)

import os
import imp
import sys
import h5py
import yaml
import numpy as np
from numba import njit

import fastpli.simulation
import fastpli.analysis
import fastpli.tools
import fastpli.io
imp.reload(fastpli)

import matplotlib.pyplot as plt
import tikzplotlib

import vector_field_generation as vfg
imp.reload(vfg)

np.random.seed(42)

FILE_PATH = os.path.abspath('')
print(FILE_PATH)
print(fastpli.__version__)

NUM_PROC = 16
with open(os.path.join(FILE_PATH, '.numba_config.yaml'), 'w') as file:
    yaml.dump({'num_threads': NUM_PROC}, file)
print(f'NUM_PROC: {NUM_PROC}')

In [ ]:
voxel_size = 0.1
scale = 10

fig, axs = plt.subplots(5, 3,figsize=(10, 10))
for m, mode in enumerate(["NN", "Lerp", "Slerp"]):
    
    simpli = fastpli.simulation.Simpli()
    simpli.omp_num_threads = NUM_PROC
    print(FILE_PATH)
    simpli.fiber_bundles = fastpli.io.fiber_bundles.load(
        os.path.join(FILE_PATH, '..', 'data', 'models', '1_rnd_seed',
                     'cube_2pop_psi_1.00_omega_0.00_r_1.00_v0_210_.solved.h5'))

    simpli.voxel_size = voxel_size  # in µm meter
#     simpli.set_voi([-5] * 3, [5] * 3)  # in µm meter
    simpli.set_voi([-10] * 3, [10] * 3)  # in µm meter
    voxel_size_0 = simpli.voxel_size
    
    simpli.fiber_bundles_properties = [[(1.0, 0.002, 0, 'r')]
                                       ] * len(simpli.fiber_bundles)
    if m == 0:
        # low resolution
        simpli.voxel_size = voxel_size_0  # in µm meter
        print(
            f"low res: {scale}, mode: {mode} -> {simpli.memory_usage('MB'):.0f} MB"
        )

        tissue, optical_axis, tissue_properties = simpli.generate_tissue()
        
        # high resolution
        simpli.voxel_size = voxel_size_0 / scale  # in µm meter
        print(
            f"scale: {scale}, mode: {mode} -> {simpli.memory_usage('MB'):.0f} MB"
        )
        if simpli.memory_usage('MB') > 128 / 3 * 1e3:
            print("MEMORY!")
            sys.exit(1)
        tissue_high, optical_axis_high, tissue_properties = simpli.generate_tissue(
        )
        
        mask = optical_axis_high[:,:,:,0] < 0
        optical_axis_high[mask,:] *= 1
        axs[m, 0].imshow(optical_axis_high[:, tissue_high.shape[1] // 2, :,0],
                     vmin=-1,
                     vmax=1, interpolation='none')
        axs[m, 1].imshow(optical_axis_high[:, tissue_high.shape[1] // 2, :,1],
                         vmin=-1,
                         vmax=1, interpolation='none')
        axs[m, 2].imshow(optical_axis_high[:, tissue_high.shape[1] // 2, :,2],
                         vmin=-1,
                         vmax=1, interpolation='none')
        
    print(f"IntpVecField: {mode}")
    tissue_int = np.empty_like(tissue_high)
    vf_intp = np.empty_like(optical_axis_high)

    simpli._Simpli__sim.__field_interpolation(tissue.shape, tissue_int.shape,
                                              tissue, optical_axis, tissue_int,
                                              vf_intp, mode)
    
    mask = vf_intp[:,:,:,0] < 0
    vf_intp[mask,:] *= 1

    axs[m+1, 0].imshow(vf_intp[:, tissue_high.shape[1] // 2, :,0],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m+1, 1].imshow(vf_intp[:, tissue_high.shape[1] // 2, :,1],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m+1, 2].imshow(vf_intp[:, tissue_high.shape[1] // 2, :,2],
                     vmin=-1,
                     vmax=1, interpolation='none')
    
    print("diff")
#     vf_diff = vfg.VectorOrientationDiffNorm(optical_axis_high, vf_intp)
    vf_diff = vfg.VectorOrientationDiffAngle(optical_axis_high, vf_intp)
    vf_diff = np.rad2deg(vf_diff)
#     vmax = np.amax(vf_diff)
#     print(f"vmax: {vmax}")
#     vmax = np.pi/2
    
    # angle diff
    tmp = np.logical_or(np.logical_and(tissue_high > 0, tissue_high % 2 == 0),
                        vf_diff != 0)
    tmp = vf_diff[tmp].ravel()

    # sort plot
    tmp.sort()
    if tmp.size > 1000:
        s = tmp.size // 1000
        tmp = tmp[::s]

    x = np.arange(tmp.size) / tmp.size
    
    x = x[tmp>0]
    tmp = tmp[tmp>0]
    x = x[tmp<1]
    tmp = tmp[tmp<1]
    axs[4,1].plot(x, tmp, label=mode)
    
#     # norm diff
#     vf_diff = vfg.VectorOrientationDiffNorm(optical_axis_high, vf_intp)
#     tmp = np.logical_or(np.logical_and(tissue_high > 0, tissue_high % 2 == 0),
#                         vf_diff != 0)
#     tmp = vf_diff[tmp].ravel()
#     # sort plot
#     tmp.sort()
#     if tmp.size > 1000:
#         s = tmp.size // 1000
#         tmp = tmp[::s]
    
axs[4,1].legend()


    

In [ ]:
voxel_size = 0.1
scale = 3

fig, axs = plt.subplots(3, 12,figsize=(20, 20))
for m, mode in enumerate(["NN", "Lerp", "Slerp"]):
    
    simpli = fastpli.simulation.Simpli()
    simpli.omp_num_threads = NUM_PROC
    print(FILE_PATH)
    simpli.fiber_bundles = fastpli.io.fiber_bundles.load(
        os.path.join(FILE_PATH, '..', 'data', 'models', '1_rnd_seed',
                     'cube_2pop_psi_1.00_omega_0.00_r_1.00_v0_210_.solved.h5'))

    simpli.voxel_size = voxel_size  # in µm meter
    simpli.set_voi([-5] * 3, [5] * 3)  # in µm meter
    # simpli.set_voi([-30] * 3, [30] * 3)  # in µm meter
    voxel_size_0 = simpli.voxel_size
    
    simpli.fiber_bundles_properties = [[(0.75, 0, 0, 'b'), (1.0, 0.002, 0, 'r')]
                                       ] * len(simpli.fiber_bundles)

    # low resolution
    simpli.voxel_size = voxel_size_0  # in µm meter
    print(
        f"low res: {scale}, mode: {mode} -> {simpli.memory_usage('MB'):.0f} MB"
    )
    tissue, optical_axis, tissue_properties = simpli.generate_tissue()

    # high resolution
    simpli.voxel_size = voxel_size_0 / scale  # in µm meter
    print(
        f"scale: {scale}, mode: {mode} -> {simpli.memory_usage('MB'):.0f} MB"
    )
    if simpli.memory_usage('MB') > 128 / 3 * 1e3:
        print("MEMORY!")
        sys.exit(1)
    tissue_high, optical_axis_high, tissue_properties = simpli.generate_tissue(
    )
    print("IntpVecField")

    tissue_int = np.empty_like(tissue_high)
    vf_intp = np.empty_like(optical_axis_high)

    simpli._Simpli__sim.__field_interpolation(tissue.shape, tissue_int.shape,
                                              tissue, optical_axis, tissue_int,
                                              vf_intp, mode)
    #     vf_intp = vfg.IntpVecField(tissue, optical_axis, scale, True, pool)
    print("diff")
#     vf_diff = vfg.VectorOrientationDiffNorm(optical_axis_high, vf_intp)
    vf_diff = vfg.VectorOrientationDiffAngle(optical_axis_high, vf_intp)
    vf_diff = np.rad2deg(vf_diff)
    vmax = np.amax(vf_diff)
    print(f"vmax: {vmax}")
    vmax = np.pi/2
    axs[m, 0].imshow(vf_diff[vf_diff.shape[0] // 2, :, :],
                     vmin=0,
                     vmax=vmax, interpolation='none')
    axs[m, 1].imshow(vf_diff[:, vf_diff.shape[1] // 2, :],
                     vmin=0,
                     vmax=vmax, interpolation='none')
    pcm = axs[m, 2].imshow(vf_diff[:, :, vf_diff.shape[2] // 2],
                           vmin=0,
                           vmax=vmax, interpolation='none')
    axs[m, 4].imshow(optical_axis_high[:, vf_diff.shape[1] // 2, :,0],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m, 5].imshow(optical_axis_high[:, vf_diff.shape[1] // 2, :,1],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m, 6].imshow(optical_axis_high[:, vf_diff.shape[1] // 2, :,2],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m, 7].imshow(vf_intp[:, vf_diff.shape[1] // 2, :,0],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m, 8].imshow(vf_intp[:, vf_diff.shape[1] // 2, :,1],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m, 9].imshow(vf_intp[:, vf_diff.shape[1] // 2, :,2],
                     vmin=-1,
                     vmax=1, interpolation='none')
    axs[m, 10].imshow(np.linalg.norm(optical_axis_high,axis=-1)[:, vf_diff.shape[1] // 2, :],
                     vmin=0,
                     vmax=1, interpolation='none')
    axs[m, 11].imshow(np.linalg.norm(vf_intp,axis=-1)[:, vf_diff.shape[1] // 2, :],
                     vmin=0,
                     vmax=1, interpolation='none')
    
    # angle diff
    tmp = np.logical_or(np.logical_and(tissue_high > 0, tissue_high % 2 == 0),
                        vf_diff != 0)
    tmp = vf_diff[tmp].ravel()

    # sort plot
    tmp.sort()
    tmp = tmp[tmp>0]
    tmp = tmp[tmp<1]
    if tmp.size > 1000:
        s = tmp.size // 1000
        tmp = tmp[::s]

    x = np.arange(tmp.size) / tmp.size
    axs[1,3].plot(x, tmp, label=mode)
    
    # norm diff
    vf_diff = vfg.VectorOrientationDiffNorm(optical_axis_high, vf_intp)
    tmp = np.logical_or(np.logical_and(tissue_high > 0, tissue_high % 2 == 0),
                        vf_diff != 0)
    tmp = vf_diff[tmp].ravel()
    # sort plot
    tmp.sort()
    if tmp.size > 1000:
        s = tmp.size // 1000
        tmp = tmp[::s]

    x = np.arange(tmp.size) / tmp.size
    axs[2,3].plot(x, tmp, label=mode)
axs[1,3].legend()
axs[2,3].legend()

In [ ]:
# show optical_axis_high_norm
# vf_norm = np.linalg.norm(optical_axis_high, axis=-1)
fig, axs = plt.subplots(1, 3)
axs[0].imshow(np.linalg.norm(optical_axis_high[vf_diff.shape[0] // 2, :, :], axis=-1), vmin=0, vmax=1)
axs[1].imshow(np.linalg.norm(optical_axis_high[:, vf_diff.shape[1] // 2, :], axis=-1), vmin=0, vmax=1)
axs[2].imshow(np.linalg.norm(optical_axis_high[:, :, vf_diff.shape[2] // 2], axis=-1), vmin=0, vmax=1)

os.makedirs(os.path.join(FILE_PATH, "results"), exist_ok=True)
tikzplotlib.save(
    os.path.join(FILE_PATH, "results", f"optical_axis_high_norm_{voxel_size:.02f}_{scale}_{model}.tex"), tex_relative_path_to_data="\currfiledir")

In [ ]:
# show vf+intp
# vf_norm_ = np.linalg.norm(vf_intp, axis=-1)
fig, axs = plt.subplots(1, 3)
axs[0].imshow(np.linalg.norm(vf_intp[vf_diff.shape[0] // 2, :, :], axis=-1), vmin=0, vmax=1)
axs[1].imshow(np.linalg.norm(vf_intp[:, vf_diff.shape[1] // 2, :], axis=-1), vmin=0, vmax=1)
axs[2].imshow(np.linalg.norm(vf_intp[:, :, vf_diff.shape[2] // 2], axis=-1), vmin=0, vmax=1)

os.makedirs(os.path.join(FILE_PATH, "results"), exist_ok=True)
tikzplotlib.save(
    os.path.join(FILE_PATH, "results", f"vf_intp_norm_{voxel_size:.02f}_{scale}_{model}.tex"), tex_relative_path_to_data="\currfiledir")

In [ ]:
# # show tissue diff
# tmp = (vf_norm != 0) * 1 + (vf_norm_ != 0) * 2
# fig, axs = plt.subplots(1, 3)
# axs[0].imshow(tmp[tmp.shape[0] // 2, :, :], vmin=0, vmax=3)
# axs[1].imshow(tmp[:, tmp.shape[1] // 2, :], vmin=0, vmax=3)
# axs[2].imshow(tmp[:, :, tmp.shape[2] // 2], vmin=0, vmax=3)

# os.makedirs(os.path.join(FILE_PATH, "results"), exist_ok=True)
# tikzplotlib.save(os.path.join(FILE_PATH, "results",
#                               f"tmp_{voxel_size:.02f}_{scale}_{model}.tex"))

In [ ]:
# # show vf_diff
# fig, axs = plt.subplots(1, 3)
# axs[0].imshow(vf_diff[vf_diff.shape[0] // 2, :, :], vmin=0, vmax=vmax)
# axs[1].imshow(vf_diff[:, vf_diff.shape[1] // 2, :], vmin=0, vmax=vmax)
# axs[2].imshow(vf_diff[:, :, vf_diff.shape[2] // 2], vmin=0, vmax=vmax)

# os.makedirs(os.path.join(FILE_PATH, "results"), exist_ok=True)
# tikzplotlib.save(
#     os.path.join(FILE_PATH, "results", f"vf_diff_{voxel_size:.02f}_{scale}_{model}.tex"), tex_relative_path_to_data="\currfiledir")

In [ ]:
# show vf_diff with tissue_high overlay
fig, axs = plt.subplots(1, 3, frameon=False)
axs[0].imshow(vf_diff[vf_diff.shape[0] // 2, :, :], vmin=0, vmax=vmax)
axs[1].imshow(vf_diff[:, vf_diff.shape[1] // 2, :], vmin=0, vmax=vmax)
axs[2].imshow(vf_diff[:, :, vf_diff.shape[2] // 2], vmin=0, vmax=vmax)
# tis = tissue_high.copy()
# print(np.sum(tis == 4))
# tis[tis % 2 == 1] = 0
axs[0].imshow(tissue_high[vf_diff.shape[0] // 2, :, :],
              vmin=0,
              vmax=4,
              cmap='gray',
              alpha=0.25)
axs[1].imshow(tissue_high[:, vf_diff.shape[1] // 2, :],
              vmin=0,
              vmax=4,
              cmap='gray',
              alpha=0.25)
axs[2].imshow(optical_axis_high[:, :, vf_diff.shape[2] // 2],
              vmin=0,
              vmax=4,
              cmap='gray',
              alpha=0.25)
for ax in axs:
    ax.axis('off')

# print(os.path.join(FILE_NAME, "results"))

os.makedirs(os.path.join(FILE_PATH, "results"), exist_ok=True)
tikzplotlib.save(
    os.path.join(FILE_PATH, "results", f"vdiff_{voxel_size:.02f}_{scale}_{model}.tex"), tex_relative_path_to_data="\currfiledir")

In [ ]:
# phi = np.linspace(-0.1 * np.pi, 1.1 * np.pi, 100, True)
# theta = phi * 0
# v = np.array(
#     np.array([
#         np.sin(phi) * np.cos(theta),
#         np.cos(phi) * np.cos(theta),
#         np.sin(theta)
#     ]).T)
# v0 = np.zeros_like(v)
# v0[:, 0] = 1
# vf_difff = np.linalg.norm(vfg.VectorOrientationSubstractionField(v0, v),
#                           axis=-1)

# plt.plot(phi / np.pi * 180, vf_difff)
# print("WISO > 1 ??")